## Lab 1: End to End Machine Learning Process

> 2019707044 전영우 😁

- Explanation
    - Rewrite the existing code 'Handson_02_end_to_end_machine_learning_project_rev1' without unnecessary codes. 
    
- Object
    - Rewrite the code without using Pipeline() object
    - Remove all unnecessary parts of the code to create a single executable code
    - Try to improve the performance of the model 
    
- Skills
    - Python
    - Numpy, Pandas, Matplotlib
    - Other Machine Learining Libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
import tarfile
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

class CombinedAttributesAdder(): # fit과 transform 메서드를 정의해야 함 -> Pipeline에 사용
    def fit(self, X, y=None): # 아무것도 안 함
        return self  
    def transform(self, X): # 특징을 유의미한 값으로 변환 -> 구 당 평균을 가구 당 평균으로
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]

        X = np.delete(X, [households_ix, rooms_ix, population_ix, bedrooms_ix], 1)

        return np.c_[X, rooms_per_household, population_per_household,
                    bedrooms_per_room]

                    
'''데이터셋 가져오기'''
url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
urllib.request.urlretrieve(url, "housing.tgz")
tar = tarfile.open("housing.tgz")
tar.extractall()
tar.close()
housing = pd.read_csv("housing.csv")

'''데이터셋 분리'''
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

'''데이터 전처리'''
housing = strat_train_set.drop("median_house_value", axis=1)   # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()  # 집값이 예측값

# col_names = ["total_rooms", "total_bedrooms", "population", "households"] # 한 구역의 total 값은 예측에 도움이 안 됨
# rooms_ix, bedrooms_ix, population_ix, households_ix = [
#     housing.columns.get_loc(c) for c in col_names] # get the column indices

# attr_adder = CombinedAttributesAdder()

'''
using Pipeline!
여러 단계의 전처리를 하나의 Pipeline 인스턴스로 생성
- 마지막 인스턴스를 제외하고는 모두 fit과 transform 메서드를 가져야 함
- 마지막 인스턴스는 fit 메서드를 가져야 함
'''
# numerical data애 댜헌 파이프라인
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # Imputing
        ('std_scaler', StandardScaler()),              # Scaling
    ])

housing_num = housing.drop("ocean_proximity", axis=1)

num_attribs = list(housing_num)   # numerical data 
cat_attribs = ["ocean_proximity"] # categorical data

# 최종 전체 파이프라인
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),    # numerical data는 num_pipeline 사용
        ("cat", OneHotEncoder(), cat_attribs), # categorical data는 onehotencoding 사용
    ])

housing_prepared = full_pipeline.fit_transform(housing)

'''모델 훈련'''
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_r2 = r2_score(housing_labels, housing_predictions)
print("Performance for Train dataset: ", lin_mse, np.sqrt(lin_mse), lin_r2)

'''모델 테스트(성능 평가)'''
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = lin_reg.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
print("Final performance evaluation: ", final_mse, np.sqrt(final_mse), lin_reg.score(X_test_prepared, y_test))

Performance for Train dataset:  4690206849.028331 68485.08486545323 0.6474749790170282
Final performance evaluation:  4732590772.51367 68793.82801177494 0.6457397959248092


## Setup

Import necessary libraries. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# import urllib.request
# import tarfile
from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Import dataset

Import [California housing dataset](https://www.kaggle.com/camnugent/california-housing-prices) and convert it to pandas dataframe format

In [119]:
# url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
# urllib.request.urlretrieve(url, "housing.tgz")
# tar = tarfile.open("housing.tgz")
# tar.extractall()
# tar.close()
housing = pd.read_csv("housing.csv")

housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


## Data Analysis

In [120]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [121]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


## Split dataset to Training/Validation Set

In [122]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    train_set = housing.loc[train_index]
    test_set = housing.loc[test_index]
    

In [123]:
train_feature, train_label = train_set.drop('median_house_value',axis=1), train_set['median_house_value']

In [124]:
train_feature.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat
17080,-122.19,37.48,38.0,1300.0,269.0,608.0,292.0,4.5568,NEAR BAY,4
7348,-118.16,33.97,39.0,1444.0,447.0,1890.0,416.0,2.1181,<1H OCEAN,2
6188,-117.92,34.10,35.0,2994.0,603.0,1933.0,561.0,4.0052,<1H OCEAN,3
13945,-117.20,34.26,17.0,9419.0,1455.0,1382.0,459.0,6.2233,INLAND,5
13656,-117.30,34.05,6.0,2155.0,NaN,1039.0,391.0,1.6675,INLAND,2
19932,-119.33,36.30,11.0,3045.0,NaN,1563.0,516.0,5.4337,INLAND,4
20007,-119.03,36.13,24.0,2259.0,408.0,1169.0,395.0,1.7106,INLAND,2
9502,-123.54,39.17,18.0,2251.0,510.0,1032.0,369.0,2.2946,<1H OCEAN,2
1987,-119.81,36.73,47.0,1314.0,416.0,1155.0,326.0,1.3720,INLAND,1
13448,-117.41,34.10,5.0,4937.0,1139.0,2204.0,812.0,2.5272,INLAND,2


In [126]:
test_feature, test_label = test_set.drop('median_house_value',axis=1), test_set['median_house_value']

## Data Preprocessing

강의 내용과 다르게, 하나의 'district'에 대한 데이터를 하나의 'house'에 대한 데이터로 변환하지 않는다. 

이는 샘플이 하나의 'district' 단위이기 때문에 가구 당 평균치로 환산하는 것이 오히려 이 데이터에서는 성능을 떨어트릴 수 있기 때문이다. 

대신, NaN 값이 있는 'total bedroom' feature의 값을 대체하고, categorical data인 'ocean proximity'의 값을 one-hot encoding한다. 

### Imputing

'total_bedrooms' feature가 결측치를 갖기 때문에 상관도가 높고 결측치가 없는 다른 특성을 이용해 결측치를 채운다. 

In [131]:
train_set.corr()['total_bedrooms'].sort_values(ascending=False)

total_bedrooms        1.000000
households            0.977683
total_rooms           0.929195
population            0.870117
longitude             0.062633
median_house_value    0.047338
median_income        -0.011084
latitude             -0.060123
housing_median_age   -0.316702
Name: total_bedrooms, dtype: float64

In [113]:
class MyImputer():
    def __init__(self):
        self.proportion = 0
        
    def fit(self,features,labels,reset=True):
        tot_feature, tot_label = 0, 0
        for feature,label in zip(features,labels):
            if not np.isnan(feature) and not np.isnan(label):  
                tot_feature += feature
                tot_label += label
                
        if reset: self.proportion = tot_feature / tot_label
        else: self.proportion = (tot_feature / tot_label + self.proportion) / 2
        
        return
        
    def transform(self,features,labels):
        imputed_features = []
        for feature,label in zip(features,labels):
            if np.isnan(feature) and not np.isnan(label):
                imputed_features.append(round(label * self.proportion))
            else:
                imputed_features.append(feature)
                
        return imputed_features
                
    def fit_transform(self,features,labels,reset=True):
        self.fit(features,labels,reset)
        return self.transform(features,labels)

In [132]:
imputer = MyImputer()

# correlation이 높은 feature를 사용해 결측치 보간
label_feature = train_set.corr()['total_bedrooms'].sort_values(ascending=False)[1]
train_feature['total_bedrooms'] = imputer.fit_transform(train_feature['total_bedrooms'], train_label)

In [133]:
print(sum(train_feature['total_bedrooms'].isna()))

train_feature.head(10)

0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat
17080,-122.19,37.48,38.0,1300.0,269.0,608.0,292.0,4.5568,NEAR BAY,4
7348,-118.16,33.97,39.0,1444.0,447.0,1890.0,416.0,2.1181,<1H OCEAN,2
6188,-117.92,34.10,35.0,2994.0,603.0,1933.0,561.0,4.0052,<1H OCEAN,3
13945,-117.20,34.26,17.0,9419.0,1455.0,1382.0,459.0,6.2233,INLAND,5
13656,-117.30,34.05,6.0,2155.0,249.0,1039.0,391.0,1.6675,INLAND,2
19932,-119.33,36.30,11.0,3045.0,348.0,1563.0,516.0,5.4337,INLAND,4
20007,-119.03,36.13,24.0,2259.0,408.0,1169.0,395.0,1.7106,INLAND,2
9502,-123.54,39.17,18.0,2251.0,510.0,1032.0,369.0,2.2946,<1H OCEAN,2
1987,-119.81,36.73,47.0,1314.0,416.0,1155.0,326.0,1.3720,INLAND,1
13448,-117.41,34.10,5.0,4937.0,1139.0,2204.0,812.0,2.5272,INLAND,2


### One-Hot Encoding